
Dataset Information

The "spam" category encompasses various types of unsolicited messages, including advertisements for products or websites, schemes promising quick financial gains, chain letters, and content related to pornography. The SMS Spam Collection dataset comprises a collection of SMS messages in English, totaling 5,574 messages. These messages have been specifically gathered for research on SMS spam. Each message in the dataset is labeled as either "ham," indicating it is legitimate, or "spam," indicating it is unsolicited or unwanted.

In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

In [3]:
df = pd.read_csv(r'C:\\Users\\MALATHIKA.P\\Downloads\\spam.csv', encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
# In order to resolve error while loading dataset, suitable encoding is checked
# Try reading with different encodings
encodings = ['utf-8', 'latin1', 'ISO-8859-1', 'windows-1252']
for encoding in encodings:
    try:
        df = pd.read_csv(r'C:\\Users\\MALATHIKA.P\\Downloads\\spam.csv', encoding=encoding)
        print("File read successfully with encoding:", encoding)
        print(df.head())
        break
    except Exception as e:
        print("Error with encoding", encoding, ":", e)

Error with encoding utf-8 : 'utf-8' codec can't decode bytes in position 135-136: invalid continuation byte
File read successfully with encoding: latin1
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [5]:
# fetching important columns for processing
df = df[['v2', 'v1']]
df = df.rename(columns={'v2': 'messages', 'v1': 'label'})
df.head()

,messages,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


Preprocessing the dataset

In [6]:
# checking for null values
df.isnull().sum()

messages    0
label       0
dtype: int64

Variables Splitting

In [7]:
import nltk
from nltk.corpus import stopwords

# download stopwords if not already
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MALATHIKA.P\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    # removing special characters
    text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    # removing extra spaces
    text = re.sub(r'\s+', ' ', text)
    # removing stopwords
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [11]:
# cleaning the messages
df['clean_text'] = df['messages'].apply(clean_text)
df.head()

,messages,label,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,ham,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah think goes usf lives around though


Variables Splitting

X = df['clean_text']
y = df['label']

Model Training

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

def classify(model, X, y):
    # train test split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', model)])
    pipeline_model.fit(x_train, y_train)
    
    print('Accuracy:', pipeline_model.score(x_test, y_test)*100)
    
#     cv_score = cross_val_score(model, X, y, cv=5)
#     print("CV Score:", np.mean(cv_score)*100)
    y_pred = pipeline_model.predict(x_test)
    print(classification_report(y_test, y_pred))

In [ ]:
Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model, X, y)

Accuracy: 96.8413496051687
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1206
        spam       0.99      0.77      0.87       187

    accuracy                           0.97      1393
   macro avg       0.98      0.88      0.92      1393
weighted avg       0.97      0.97      0.97      1393



Naive Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
classify(model, X, y)

Accuracy: 96.69777458722182
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1206
        spam       1.00      0.75      0.86       187

    accuracy                           0.97      1393
   macro avg       0.98      0.88      0.92      1393
weighted avg       0.97      0.97      0.96      1393



Support Vector Classifier

In [24]:
from sklearn.svm import SVC
model = SVC(C=3)
classify(model, X, y)

Accuracy: 98.27709978463747
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1206
        spam       1.00      0.87      0.93       187

    accuracy                           0.98      1393
   macro avg       0.99      0.94      0.96      1393
weighted avg       0.98      0.98      0.98      1393



Random Forest Classifier

In [25]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classify(model, X, y)

Accuracy: 97.63101220387652
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      1206
        spam       1.00      0.82      0.90       187

    accuracy                           0.98      1393
   macro avg       0.99      0.91      0.94      1393
weighted avg       0.98      0.98      0.98      1393

